In [ ]:
#default_exp topo_solvers

In [ ]:
#export
import time
import torch
from collections import defaultdict

from dl4to.solution import Solution
from dl4to.criteria import VolumeFraction, Binariness

In [ ]:
#hide
from nbdev.showdoc import show_doc

# SIMP iterator

In [ ]:
#export
class SIMPIterator:
    """
    Performs the actual SIMP optimization.
    """
    def __init__(
        self,
        problem:"dl4to.problem.Problem", # The problem that should be solved by SIMP.
        criterion:"dl4to.criteria.Criterion", # The objective function that should be optimized for in the optimization process.
        density_representer:"dl4to.density_representers.DensityRepresenter", # The density representer that is used for the latent density representation. The density representer also performs the projection, smoothing and filtering.
        lr:float, # The learning rate of the `torch.optim.Adam` optimizer.
        binarizer_steepening_factor:float # The factor at which the binarizer should be steepened in each iteration. The default value of 1.1 corresponds to a steepening of 10%.
    ):
        self.lr = lr
        self.logs = defaultdict(list)
        self.binarizer_steepening_factor = binarizer_steepening_factor
        self.problem = problem

        self.criterion = criterion
        self.volume_crit = VolumeFraction()
        self.binariness_crit = Binariness()
        self.density_representer = density_representer
        self.optimizer = torch.optim.Adam(self.density_representer.parameters(), lr=self.lr)


    def _extend_logs(self, solution, loss, volume, tick, σ_vm):
        self.logs["losses"].append(loss.item())
        self.logs["volumes"].append(volume.item())
        self.logs["durations"].append(time.time() - tick)
        self.logs["binarinesses"].append(self.binariness_crit([solution]))
        self.logs["relative_max_σ_vm"].append(σ_vm.max().item() / self.problem.σ_ys)


    def _perform_optimizer_step(self, loss):
        self.optimizer.zero_grad()
        loss.backward(retain_graph=True)
        grads = torch.autograd.grad(loss, self.density_representer.θ)[0].detach()
        self.optimizer.step()
        return grads


    def __call__(self, 
                 p:float # The SIMP exponent
                ):
        """
        Creates the SIMP solution objects, solves the PDE and communicates with the density representer.
        Returns a `dl4to.solution.Solution` object.
        """
        tick = time.time()
        solution = Solution(
            problem=self.problem,
            θ=self.density_representer(),
        )

        u, σ, σ_vm = solution.solve_pde(p=p)
        loss = self.criterion([solution])
        volume = self.volume_crit([solution])

        grads = self._perform_optimizer_step(loss)
        solution.θ = self.density_representer()
        self.density_representer.steepen_binarizer(self.binarizer_steepening_factor)

        self._extend_logs(solution, loss, volume, tick, σ_vm)
        solution.logs = self.logs

        solution.logs['grads'].append(grads)
        return solution

In [ ]:
#hide
from dl4to.pde import FDM
from dl4to.datasets import BasicDataset
from dl4to.criteria import Compliance, VolumeFraction, VolumeConstraint, StressConstraint

In [ ]:
#hide
μ = 1e-13
λ = 1e-5
threshold_fct = 'relu'
max_volume_fraction = 0.1
criterion = VolumeFraction() + μ * StressConstraint(threshold_fct=threshold_fct)
#criterion = Compliance() + μ * StressConstraint(threshold_fct=threshold_fct) + λ * VolumeConstraint(max_volume_fraction=max_volume_fraction)

In [ ]:
%%time
#hide

n_iterations=100
verbose=True

from dl4to.datasets import SeltoDataset
dataset = SeltoDataset("/localdata/dl4to_datasets/", name="disc_simple", train=True)
#problem = BasicDataset(resolution=40).ledge(force_per_area=-2e6)
problem = dataset[0][0]
problem.pde_solver = FDM()
density_representer = FilteringDensityRepresenter(filter_size=3, filter_fct="max_pool")

simp = SIMP(
    criterion=criterion,
    density_representer=density_representer,
    n_iterations=n_iterations,
    lr=1e-1,
    verbose=True,
    return_intermediate_solutions=True,
)

solutions = simp(problems_or_solutions=problem)


if verbose:
    solutions[-1].plot()